In [3]:
ls = sc.textFile('data/coupon150720.csv')
# para meter el número de particiones que queremos hacer, introducimos el número de particiones que queremos hacer
# al hacer la lectura del fichero:
# ls = sc.textFile('data/coupon150720.csv',100)  - > le hemos metido 100 particiones
# el resto de código sería el mismo, que si dejamos las particiones por defecto (en este caso 3 -> lo calculamos 
# más abajo)

In [4]:
# cogemos 5 líneas aleatorias
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [5]:
# cogemos la primera línea (aunque esté distribuido el fichero en un cluster)
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [6]:
#contamos las líneas del fichero
ls.count()

1232662

In [8]:
ls.cache()
# por defecto particiona en memoria
ls.persist()
# admite argumentos. Si no introduces argumento, hace lo mismo que cache, particiona en memoria

MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:-2

In [9]:
ls.getNumPartitions()
# calculamos el número de particiones que ha hecho spark por defecto

3

## Calculamos el número de pasajeros total de vuelos que ha operado Iberia (no que haya vendido) -> campo 5 de nuestros datos

In [25]:
# tenemos que partir las líneas
ls.filter(lambda x: (x.split(',')[5]=='IB'))

PythonRDD[17] at RDD at PythonRDD.scala:43

In [41]:
ls.filter(lambda x: (x.split(',')[5]=='IB')).take(5)

[u'79062005408888,1,ALC,MAD,IB,IB,313.99,USD,1,Y,Y,8393,150719,OK,INH0',
 u'79062005408888,2,MAD,TLS,IB,IB,865.32,USD,1,Y,Y,8770,150719,OK,INA0',
 u'79062006136152,1,SVQ,MAD,IB,IB,16.58,USD,1,Q,Q,3947,150910,OK,INJ0',
 u'79062006136152,4,MAD,SVQ,IB,IB,15.93,USD,2,O,O,3954,150921,OK,INJ0',
 u'79062005962805,1,SCQ,MAD,IB,IB,8.98,USD,1,S,S,3895,151001,OK,INJ0']

In [42]:
ls.filter(lambda x: (x.split(',')[5]=='IB')).count()

26158

In [43]:
# solución del profesor:
aerolineas = ls.map(lambda x: x[5])  # así nos estamos quedando con la 5ª letra

In [44]:
aerolineas.take(5)

[u'0', u'0', u'0', u'6', u'6']

In [45]:
# separamos los datos para quedarnos con la 5ª columna:
aerolineas = ls.map(lambda x: x.split(",")[5])

In [46]:
aerolineas.take(5)

[u'9W', u'9W', u'9W', u'9W', u'9W']

In [47]:
aerolineas.filter(lambda x: x=="IB").count()

26158

In [48]:
# otra opción encadenando funciones:
ls.map(lambda x: x.split(",")[5]).filter(lambda x: x == "IB").count()

26158

## vamos a ver ahora cuánto dinero han supuesto estos viajes operados por Iberia

In [50]:
# definimos una función porque con lambda nos saldría un chorizo de código
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])   # tenemos que pasar a valor la columna 6, que está como texto
                               # cuando trabajemos con muchos datos hay que tener en cuenta que puede que algún valor 
                               # no venga como texto o número (por ejemplo, vienen guiones ---)
    return (aerolinea, dinero)

In [51]:
getAerolineaDinero

<function __main__.getAerolineaDinero>

In [54]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [55]:
dineros = ls.map(getAerolineaDinero)

In [56]:
dineros.take(5)

[(u'9W', 56.79),
 (u'9W', 84.34),
 (u'9W', 60.0),
 (u'9W', 160.63),
 (u'9W', 152.46)]

In [57]:
dineros.filter(lambda x: x[0] == 'IB').take(3)

[(u'IB', 313.99), (u'IB', 865.32), (u'IB', 16.58)]

In [58]:
dinerosIB = dineros.filter(lambda x: x[0] == 'IB')

In [59]:
dinerosIB.take(3)

[(u'IB', 313.99), (u'IB', 865.32), (u'IB', 16.58)]

In [61]:
# vamos a quedarnos sólo con los valores:
cantidades = dinerosIB.map(lambda x: x[1])

In [69]:
cantidades.take(5)

[313.99, 865.32, 16.58, 15.93, 8.98]

In [70]:
# calculamos la suma y la media:
cantidades.sum()

2828044.4199999194

In [71]:
cantidades.mean()

108.11393913907767

In [72]:
# calculamos varias estadísticas
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

In [73]:
# para calcular la suma podemos multiplicar el total por la media
cantidades.count()*cantidades.mean()

2828044.419999994

In [74]:
# hacemos lo mismo pero en una sola línea:
dineros.filter(lambda x: x[0] =="IB").map(lambda x: x[1]).stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

## top ten de aerolineas con el billete más caro

In [76]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [93]:
def getRutaDinero(l):
    elems = l.split(",")
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino
    return (ruta, dinero)

In [94]:
# probamos si funciona la función antes de aplicarla al rdd
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [95]:
rutas = ls.map(getRutaDinero)

In [96]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [97]:
# ordenamos por cupones más caros:
rutas.sortBy(lambda x: x[1], False).take(5)  # ponemos False para que ordene de mayor a menor

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [98]:
# filtramos los cupones por debajo de 7.500$
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [99]:
# Tenemos rutas repetidas, por lo que tenemos que agrupar:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [100]:
filtrado.cache()

PythonRDD[109] at RDD at PythonRDD.scala:43

In [101]:
filtrado.groupByKey().take(3)
# agrupamos por rutas porque queremos saber las rutas más caras, no los cupones

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7f6cd8079910>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7f6cd8079d10>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7f6cd8a371d0>)]

In [102]:
filtrado.groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'SZF-IST', 289.83), (u'DME-RHO', 168.95), (u'KRK-OSL', 98.0)]

In [103]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs))

In [104]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

## Ruta con el valor medio por cupón más alto

In [111]:
filtrado.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).sortBy(lambda x: x[1], False).take(5)


[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

## Quitamos los precios fuera de la media según la categoría del pasajero

In [112]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [128]:
def getRutaDineroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems[3]
    al = elems[4]
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta, al, clase), dinero)

In [129]:
getRutaDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [130]:
import math

In [141]:
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]       # sumamos 1 al valor para no tener ceros (el log de algo y de ese algo +1 son resultado muy parecidos)
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) - 1    # le restamos 1 para volver a la misma escala que teníamos los datos puesto que antes lo hemos sumado
# hacemos la exponencial del resultado para quitar el logaritmo e interpretar los datos reales

In [142]:
valorLimite([12,3,4,1,2,4])
# comprobamos que funciona la función con una serie de valores cualesquiera

13.060852649090572

In [143]:
rutas = ls.map(getRutaDineroClase)

In [144]:
rutas.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [145]:
rutas.groupByKey().take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f6cd80b5b10>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f6cd80b5390>)]

In [146]:
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [147]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)

In [150]:
def mediaGrupo(vs):
    if(len(vs) == 0):
        return 0.0
    else:
        return sum(vs)/len(vs)
    
# hacemos esto para evitar que nos quite los cupones que tienen valor 0

In [148]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  [42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   48.67,
   42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76]),
 ((u'DPS-TMC', u'GA', u'N'), [59.15, 59.26, 62.38, 62.38])]

In [151]:
grupos.mapValues(mediaGrupo).take(5)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'DPS-TMC', u'GA', u'N'), 60.7925),
 ((u'BEG-CDG', u'JU', u'K'), 0.0),
 ((u'SAV-CLT', u'US', u'N'), 0.0),
 ((u'DUS-AMS', u'KL', u'C'), 0.0)]

In [152]:
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1], False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

Nos sale un billete de 6 mill porque puede ser que solo haya un billete con esa ruta.
Tenemos que descartar los grupos con pocos billetes (nos quedamos con los que tengan al menos 10 cupones)
porque si no hay suficientes datos no podemos hacer nada.

In [153]:
grupos = rutas.groupByKey()

In [154]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f6cd8079e10>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7f6cd8079c90>)]

In [156]:
gruposFiltrados = grupos.filter(lambda x: len(x[1]) > 10)
# nos quedamos con los grupos cuya longitud sea mayor que 10 (es decir, que tengan más de 10 cupones)

In [157]:
gruposFiltrados.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7f6cd8945090>),
 ((u'PHX-CLT', u'US', u'S'),
  <pyspark.resultiterable.ResultIterable at 0x7f6cd8945c90>)]

In [159]:
r = gruposFiltrados.mapValues(filtraCuponesExtremos).mapValues(mediaGrupo)

In [160]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

De esta manera ya nos hemos quitado el valor de 6 millones, por ejemplo.

## ¿Cuántas aerolíneas diferentes hay en este fichero de datos?

In [161]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [163]:
ls.map(lambda x: x.split(",")[4]).take(5)

[u'9W', u'9W', u'9W', u'9W', u'9W']

In [164]:
ls.map(lambda x: x.split(",")[4]).distinct().take(5)
# distinct tarda un poco en ejecutarse (es una función"cara") porque requiere ordenar los datos

[u'', u'BE', u'WN', u'JV', u'WK']

In [165]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

Tenemos 359 aerolíneas diferentes

## Vamos a ver si todas las monedas son en dólares o no

In [166]:
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

Sólo hay dólares

## Cómo exportar estos datos ya tratados para tratarlos luego en R, por ejemplo

In [167]:
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("results.csv")

Nos ha guardado un repositorio en el mismo directorio donde estamos trabajando en el que hay un fichero por cada
partición que hayamos hecho

In [ ]:
# si queremos que nos haga más particiones lo pondríamos así: 
ls.map(lambda x: x.split(",")[4]).reaprtition(100).distinct().saveAsTextFile("results.csv")